# Testing LB on FABRIC U280

This notebook helps sets up a sender and multiple receiver nodes on FABRIC such that they can communicate with a LB provisioned in FABRIC. Unlike the 'FABRIC-live-lb-tester' notebook, this one does NOT use FABNetv4Ext service - only FABNetv4, consequently the security measures are more limited, since none of this is exposed to the internet. One of the nodes is designated as a sender and the rest as worker-receivers.

See the following diagram:

<div>
    <img src="figs/live-lb.png" width=500>
</div>


## Preamble

This code should *always* be executed regardless of whether you are starting a new slice or returning to an existing slice.

In [1]:
#
# EDIT THIS
#

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_config/github_ecdsa'
signing_key = '/home/fabric/work/fabric_config/signing'

# Note for best management network IPv4 connectivity pick from
# 'UCSD', 'SRI', 'FIU' or 'TOKY' - these sites have
# IPv4. Other sites use IPv6 management and have trouble
# retrieving git-lfs artifacts.

# site_list_override = None

# if you want to force a site list instead of using random
site_list_override = ['SRI']

# # (super)core sites - should be low loss
# site_list_override = ['SALT', 'KANS', 'NEWY', 'WASH', 'LOSA', 'DALL', 'ATLA']

# high capacity sites (may have losses at high bandwidth)
# site_list_override = ['STAR', 'INDI', 'NCSA', 'TACC', 'UCSD', 'PSC']

# these we always exclude
site_exclude_list = ['EDUKY', 'EDC']

# how many workers do we want? (in addition to one sender)
number_of_workers = 1

# base distro 'ubuntu2[012]' or 'rocky[89]'
distro_name = 'ubuntu22'
distro_version = distro_name[-2:]

# map from distro to image name
images = {
    'ubuntu20': 'default_ubuntu_20',
    'ununtu21': 'default_ubuntu_21',
    'ubuntu22': 'default_ubuntu_22',
    'rocky8': 'default_rocky_8',
    'rocky9': 'default_rocky_9',
}

# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
home_location = {
    'ubunt': '/home/ubuntu',
    'rocky' : '/home/rocky'
}[distro_name[:5]]

vm_key_location = f'{home_location}/.ssh/github_ecdsa'
sign_key_location = f'{home_location}/.ssh/signing'

# worker dimensions
node_attribs = {
    'cores': 8,
    'disk': 100,
    'ram': 24,
    'image': images[distro_name]
}

# slice name
slice_name = f'{number_of_workers + 1}-node U280 LB Tester Slice using {distro_name} 1'

# url of e2sar deb. Find the appropriate version for the OS at https://github.com/JeffersonLab/E2SAR/releases
e2sar_branch = "main"
static_release_url = 'https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR' # don't need to change this
e2sar_release_artifact = 'e2sar_0.1.5_amd64.deb'
e2sar_release_ver = '0.1.5'
e2sar_release_url = 'https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb'
print(e2sar_release_url)

# this is the IPv4 address and Control Plane listening port of the LB node created by the other notebook
lb_node_ip = "10.133.10.254"
lb_cp_port = 18008
# the admin token is also generated by the U280 LB notebook
lb_admin_token = "COU7L26VIFSVWRVB13S32Z9JDC3DUU6V"

ejfat_admin_uri = f"ejfats://{lb_admin_token}@{lb_node_ip}:{lb_cp_port}/"

#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
import json
import time
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# gets prepended to site name - this network is per site
net_name_prefix = 'fabnetv4'

# this is the NIC to use
nic_model = 'NIC_Basic'

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)

def execute_single_node_on_thread(node, commands):
    # concatenate the commands using ';' and execute
    allcommands = ';'.join(commands)
    node.execute_thread(allcommands, output_file=node.get_name() + '_thread.log')

def execute_commands_on_threads(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node_on_thread(n, commands)
    else:
        execute_single_node_on_thread(node, commands)

def make_node_name(site_name, node_idx):
    return '_'.join([f"Worker{node_idx}", site_name])

def make_net_name(site_name):
    return '_'.join([net_name_prefix, site_name])

# return slice with one node on one site
def starter_slice(site_name):
    #node_name = make_node_name(site_name, 1)
    node_name = '_'.join(["Sender", site_name])
    net_name = make_net_name(site_name)

    slice = fablib.new_slice(name=slice_name)
    node = slice.add_node(name=node_name, site=site_name, **node_attribs)

    # postboot configuration is under 'post-boot' directory
    node.add_post_boot_upload_directory('post-boot','.')
    node.add_post_boot_execute(f'chmod +x post-boot/sender.sh && ./post-boot/sender.sh')
    
    # attach to network
    nic_interface = node.add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net = slice.add_l3network(name=net_name, interfaces=[nic_interface], type='IPv4')

    return slice

def add_node_to_slice(site_name, node_idx, inc, slice):

    net_name = make_net_name(site_name)

    while inc > 0:
        node_name = make_node_name(site_name, node_idx)
        node_idx += 1
        
        node = slice.add_node(name=node_name, site=site_name, **node_attribs)
    
        # postboot configuration is under 'post-boot' directory
        node.add_post_boot_upload_directory('post-boot','.')
        node.add_post_boot_execute(f'chmod +x post-boot/recver.sh && ./post-boot/recver.sh')
    
        nic_interface = node.add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
        
        # attach to a network, create network if needed
        net = slice.get_network(name=net_name)
        if net is None:
            net = slice.add_l3network(name=net_name, type='IPv4')
            
        net.add_interface(nic_interface)
        inc -= 1

    return None

def check_modify(slice, selected_site_list, nodes_in_slice, expected_to_add):

    success = True
    idx = 1
    while(expected_to_add >= idx):
        # find sliver reservation for new node
        node_sliver = slice.list_slivers(fields=['name', 'state'], 
                                         filter_function=lambda x: x['type'] == 'node' and 
                                             x['name'] == make_node_name(selected_site_list[0], nodes_in_slice + idx) and 
                                             x['state'] == 'Active')
        # if it is none - it failed
        if node_sliver is None:
            success = False
            break
        else:
            idx += 1

    return success

# until fablib fixes this
def get_management_os_interface(node) -> str or None:
        """
        Gets the name of the management interface used by the node's
        operating system. 

        :return: interface name
        :rtype: String
        """
        stdout, stderr = node.execute("sudo ip -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        stdout, stderr = node.execute("sudo ip -6 -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        return None

https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,srinivas_0000202712
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Helpers

If you ever forget which images are available, run this cell:

In [7]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

Available images are: ['default_centos8_stream', 'default_centos9_stream', 'default_centos_7', 'default_centos_8', 'default_debian_11', 'default_debian_12', 'default_fedora_39', 'default_fedora_40', 'default_freebsd_13_zfs', 'default_freebsd_14_zfs', 'default_kali', 'default_openbsd_7', 'default_rocky_8', 'default_rocky_9', 'default_ubuntu_20', 'default_ubuntu_22', 'default_ubuntu_24', 'docker_rocky_8', 'docker_rocky_9', 'docker_ubuntu_20', 'docker_ubuntu_22']


## Prepare to create a new slice (skip if exists)

In [8]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

ID,Name,Lease Expiration (UTC),Lease Start (UTC),Project ID,State
8a1bc512-463e-4e7e-963b-c7469c6b5dd8,"UDP LB Control Plane Testing with udplbd[develop], e2sar[e2sar-java] on ubuntu",2025-02-25 22:37:15 +0000,2025-02-12 20:22:31 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK
5210f023-b755-498a-ba3a-956056527da8,4-node U280 LB Tester Slice using ubuntu22 1,2025-02-24 01:30:08 +0000,2025-01-16 21:08:36 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK
33ad6642-f018-4800-b30c-2cd012e7b2e2,ERSAP-E2SAR locahost test [main] on ubuntu,2025-02-17 19:17:04 +0000,2025-02-16 19:17:04 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK


In [9]:
# Identify sites in continental US we want to use (NOOP if override is set)
lon_west=-124.3993243
lon_east=-69.9721573
candidate_sites = 7
free_nodes_worth = 3 # how many nodes worth are we looking per site

# get a list of random sites, avoiding thos on the exclude list
# unless there is an override
if site_list_override is None:
    selected_site_list = fablib.get_random_sites(count=candidate_sites, avoid=site_exclude_list,
                                            filter_function=lambda x: x['location'][1] < lon_east
                                            and x['location'][1] > lon_west 
                                            and x['cores_available'] > free_nodes_worth * node_attribs['cores']
                                            and x['ram_available'] > free_nodes_worth * node_attribs['ram'] 
                                            and x['disk_available'] > free_nodes_worth * node_attribs['disk']) 
else:
    selected_site_list = site_list_override

if selected_site_list:
    print(f'Selected sites are {selected_site_list}')
else:
    print('Unable to find a sites matching the requirements')


Selected sites are ['SRI']


## Create slice iteratively (skip if exists)

We may or may not get all the nodes we want immediately - we use iteration with slice modify to get to the max/desired number of nodes across the selected sites.

### Create Starter Slice

In [10]:
# we start by establishing a slice with one sender node at some site, we keep track which sites we failed 
# and don't try those again

keep_trying = True
succeeded = False

site_list_iter = iter(selected_site_list)
failed_sites = {}
site_name = None

while keep_trying:

    try:
        site_name = next(site_list_iter)
        print(f'Trying site {site_name} from {selected_site_list}')
        
        # define a starter slice
        slice = starter_slice(site_name)

        print(f'Submitting starter slice "{slice_name}" with sender on site {site_name}')
        slice_id = slice.submit()

        # check the state of this slice
        slices = fablib.get_slices(excludes=[], slice_id=slice_id)
        if slices[0].get_state() == 'Dead':
            print(f'Failed on site {site_name}, proceeding')
        else:
            print(f'Succeeded on site {site_name} with state {slices[0].get_state()}')
            keep_trying = False
            succeeded = True
    except StopIteration: 
        print('No more sites to look at, exiting')
        keep_trying = False
    except Exception as e:
        print(f'Unexpected exception {e}, exiting')
        keep_trying = False

if succeeded:
    print(f'Succeeded in creating a slice on {site_name}, will avoid sites {failed_sites}')
    selected_site_list = list(filter(lambda x: x not in failed_sites, selected_site_list))
    print(f'Proceeding with sites {selected_site_list}')


Retry: 5, Time: 237 sec


ID,1d8539d9-0dda-46bc-bcbb-6d8377ba61d1
Name,2-node U280 LB Tester Slice using ubuntu22 1
Lease Expiration (UTC),2025-02-17 19:48:23 +0000
Lease Start (UTC),2025-02-16 19:48:23 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
2c80deaa-412f-4b2d-89f1-7d81c3eee7b3,Sender_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.35,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.35,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
c20a0082-eae6-49d4-8905-9d3cd710ce48,fabnetv4_SRI,L3,FABNetv4,SRI,10.141.132.0/24,10.141.132.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Sender_SRI-Sender_SRI_NIC_Basic_nic-p1,p1,Sender_SRI,fabnetv4_SRI,100,config,,06:B2:92:D9:79:77,enp7s0,enp7s0,fe80::4b2:92ff:fed9:7977,6,HundredGigE0/0/0/5



Time to print interfaces 240 seconds
Succeeded on site SRI with state StableOK
Succeeded in creating a slice on SRI, will avoid sites {}
Proceeding with sites ['SRI']


### Modify the Slice to add Workers

Now that the base with the sender slice is created we will iteratively add workers on sites one at a time using first-fit policy until we get to the desired number of workers or run out of sites.

In [11]:
remaining_workers = number_of_workers
node_idx = 1
node_increment = 3
nodes_in_slice = 0 # we don't count sender in this case

while remaining_workers > 0 and len(selected_site_list) > 0:
    slice = fablib.get_slice(name=slice_name)
    
    try:
        site_name = selected_site_list[0]
        print(f'There are {remaining_workers} remaining workers to create. Trying site {site_name} from {selected_site_list}')
        expected_to_add = node_increment if remaining_workers >= node_increment else remaining_workers
        add_node_to_slice(site_name, node_idx, expected_to_add, slice)
        
        print(f'Submitting slice modification to "{slice_name}" to add {expected_to_add} nodes for site {site_name}')
        slice_id = slice.modify()
        
        # check the state of this slice
        slice = fablib.get_slice(name=slice_name)

        if check_modify(slice, selected_site_list, nodes_in_slice, expected_to_add):
            print(f'Succeeded adding {expected_to_add} nodes on site {site_name}.')
            # successfully provisioned
            node_idx += expected_to_add
            remaining_workers -= expected_to_add
            nodes_in_slice += expected_to_add
        else:
            print(f'Failed to provision on site {site_name}.')
            # this site is full, moving on
            selected_site_list.remove(site_name)            
    except Exception as e:
        remaining_workers = -1
        print(f'Unexpected exception {e}, exiting')
        break

if remaining_workers == 0:
    print('Succeeded in creating all workers')
else:
    print(f'Unable to create {remaining_workers}')



Retry: 9, Time: 231 sec


ID,1d8539d9-0dda-46bc-bcbb-6d8377ba61d1
Name,2-node U280 LB Tester Slice using ubuntu22 1
Lease Expiration (UTC),2025-02-17 19:48:23 +0000
Lease Start (UTC),2025-02-16 19:48:23 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
2c80deaa-412f-4b2d-89f1-7d81c3eee7b3,Sender_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.35,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.35,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e836a911-0a1a-466c-9adf-dde57ecb1a0f,Worker1_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
c20a0082-eae6-49d4-8905-9d3cd710ce48,fabnetv4_SRI,L3,FABNetv4,SRI,10.141.132.0/24,10.141.132.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Sender_SRI-Sender_SRI_NIC_Basic_nic-p1,p1,Sender_SRI,fabnetv4_SRI,100,config,,06:B2:92:D9:79:77,enp7s0,enp7s0,fe80::4b2:92ff:fed9:7977,6,HundredGigE0/0/0/5
Worker1_SRI-Worker1_SRI_NIC_Basic_nic-p1,p1,Worker1_SRI,fabnetv4_SRI,100,config,,06:13:45:4B:83:8B,enp7s0,enp7s0,fe80::413:45ff:fe4b:838b,6,HundredGigE0/0/0/5



Time to print interfaces 238 seconds


Name,State
Worker1_SRI,Active


Succeeded adding 1 nodes on site SRI.
Succeeded in creating all workers


## Get Slice Details (always execute)

The following code sets up data structures so all the follow up cells work properly. Execute it regardless of whether you just created the slice or coming back to an existing slice.

In [2]:
def find_net(net_list, name):
    return next(filter(lambda x: x.get_name() == name, net_list), None)

# this is the full /10 routable FABRIC IPv4
full_subnet = ipaddress.IPv4Network('10.128.0.0/10')

# get slice details 
slice = fablib.get_slice(name=slice_name)

a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

# arrange nodes and network services by site for future convenience
net_objects = slice.get_networks()
node_objects = slice.get_nodes()

slivers_by_site = dict()

print('Arranging nodes and networks by site and getting available IP addresses')
for node in node_objects:
    node_site = node.get_site()
    if not slivers_by_site.get(node_site):
        slivers_by_site[node_site] = dict()
        slivers_by_site[node_site]['nodes'] = set()
        slivers_by_site[node_site]['net'] = find_net(net_objects, make_net_name(node_site))
    slivers_by_site[node_site]['nodes'].add(node)

print('Listing IPv4 addresses per service (per site)')
for net in net_objects:
    print(f'{net.get_name()} has {net.get_subnet()}')


ID,1d8539d9-0dda-46bc-bcbb-6d8377ba61d1
Name,2-node U280 LB Tester Slice using ubuntu22 1
Lease Expiration (UTC),2025-02-17 19:48:23 +0000
Lease Start (UTC),2025-02-16 19:48:23 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
c20a0082-eae6-49d4-8905-9d3cd710ce48,fabnetv4_SRI,L3,FABNetv4,SRI,10.141.132.0/24,10.141.132.1,Active,


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
2c80deaa-412f-4b2d-89f1-7d81c3eee7b3,Sender_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.35,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.35,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e836a911-0a1a-466c-9adf-dde57ecb1a0f,Worker1_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


Arranging nodes and networks by site and getting available IP addresses
Listing IPv4 addresses per service (per site)
fabnetv4_SRI has 10.141.132.0/24


## Perform network configuration

### Set up routing

In [13]:
def get_valid_ips(ip_list, gateway, qty=1):
    # make sure the returned IP isn't the gateway
    ips = []
    while qty > 0:
        ip = ip_list.pop()
        while ip == gateway:
            ip = ip_list.pop()
        ips.append(ip)
        qty -= 1

    return ips

# allocate IP addresses in each site network services
print('Allocating IP addresses in sites where slice is present')
for site_name, site_slivers  in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    site_nodes = site_slivers['nodes']
    # this properly lists needed available IPs for this subnet
    site_slivers['ips'] = list(site_net.get_subnet().hosts())[:len(site_nodes)]
    site_slivers['ips'] = get_valid_ips(list(site_net.get_subnet().hosts()), site_net.get_gateway(), len(site_nodes))
    print(f'Using the following IPs: {site_slivers["ips"]}')

Allocating IP addresses in sites where slice is present
Processing SRI
Using the following IPs: [IPv4Address('10.141.132.254'), IPv4Address('10.141.132.253')]


In [14]:
# configure node interfaces with these IP addresses
print('Configuring interfaces on nodes')
for site_name, site_slivers in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    site_nodes = site_slivers['nodes']
    site_addrs = site_slivers['ips']
    for node, addr in zip(site_nodes, site_addrs):
        print(f'  Adding address {addr} to node {node.get_name()} in subnet {site_net.get_subnet()}')
        # make sure the interface is UP (in rare cases comes up in DOWN state)
        node_iface = node.get_interface(network_name=site_net.get_name())
        execute_single_node(node, [f'sudo ip link set {node_iface.get_os_interface()} up'])
        node_iface.ip_addr_add(addr=addr, subnet=site_net.get_subnet())
        node.ip_route_add(subnet=full_subnet, gateway=site_net.get_gateway())


Configuring interfaces on nodes
Processing SRI
  Adding address 10.141.132.254 to node Sender_SRI in subnet 10.141.132.0/24
	Executing "sudo ip link set enp7s0 up" on node Sender_SRI
  Adding address 10.141.132.253 to node Worker1_SRI in subnet 10.141.132.0/24
	Executing "sudo ip link set enp7s0 up" on node Worker1_SRI


In [15]:
# We need to setup the firewall to allow UDP traffic to pass between the nodes
# we do this by moving all interfaces (loopback, management and data) to the 'trusted' zone
# since we are in FABNetv4 we can keep things simple

for site_name, site_slivers in slivers_by_site.items():
    print(f'Processing {site_name}')
    site_net = site_slivers['net']
    for node in site_slivers['nodes']:
        mgmt_iface_name = get_management_os_interface(node)
        data_iface = node.get_interface(network_name=site_net.get_name())
        data_iface_name = data_iface.get_os_interface()
        commands = [
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface={data_iface_name}',
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface=lo',
            f'sudo firewall-cmd --permanent --zone=trusted --add-interface={mgmt_iface_name}',
            f'for i in $(sudo firewall-cmd --zone=public --list-services); do sudo firewall-cmd --zone=public --permanent --remove-service=$i; done',
        ]
        commands.append(f'sudo firewall-cmd --reload')
        commands.append(f'sudo firewall-cmd --list-all --zone=trusted')
        execute_commands([node], commands)

Processing SRI
	Executing "sudo firewall-cmd --permanent --zone=trusted --add-interface=enp7s0" on node Sender_SRI
success
	Executing "sudo firewall-cmd --permanent --zone=trusted --add-interface=lo" on node Sender_SRI
success
	Executing "sudo firewall-cmd --permanent --zone=trusted --add-interface=enp3s0" on node Sender_SRI
success
	Executing "for i in $(sudo firewall-cmd --zone=public --list-services); do sudo firewall-cmd --zone=public --permanent --remove-service=$i; done" on node Sender_SRI
success
success
	Executing "sudo firewall-cmd --reload" on node Sender_SRI
success
	Executing "sudo firewall-cmd --list-all --zone=trusted" on node Sender_SRI
trusted (active)
  target: ACCEPT
  icmp-block-inversion: no
  interfaces: enp3s0 enp7s0 lo
  sources: 
  services: 
  ports: 
  protocols: 
  forward: yes
  masquerade: no
  forward-ports: 
  source-ports: 
  icmp-blocks: 
  rich rules: 
	Executing "sudo firewall-cmd --permanent --zone=trusted --add-interface=enp7s0" on node Worker1_SRI


## Tune Buffers and MTUs

In order to have good performance we need to
- Make the UDP send/receive socket buffer size limit larger (applications are assumed to know how to make their buffers larger up to this limit)
- Set MTU to 9k and test with DF=0 ping

In [16]:
# setup UDP socket buffer sizes to 512M
commands = [
    f"sudo sysctl net.core.rmem_max=536870912",
    f"sudo sysctl net.core.wmem_max=536870912",
    f"sysctl net.core.wmem_max net.core.rmem_max"
]
# walk the nodes
for site_name, site_slivers in slivers_by_site.items():
    for node in site_slivers['nodes']:
        execute_single_node(node, commands)

	Executing "sudo sysctl net.core.rmem_max=536870912" on node Sender_SRI
net.core.rmem_max = 536870912
	Executing "sudo sysctl net.core.wmem_max=536870912" on node Sender_SRI
net.core.wmem_max = 536870912
	Executing "sysctl net.core.wmem_max net.core.rmem_max" on node Sender_SRI
net.core.wmem_max = 536870912
net.core.rmem_max = 536870912
	Executing "sudo sysctl net.core.rmem_max=536870912" on node Worker1_SRI
net.core.rmem_max = 536870912
	Executing "sudo sysctl net.core.wmem_max=536870912" on node Worker1_SRI
net.core.wmem_max = 536870912
	Executing "sysctl net.core.wmem_max net.core.rmem_max" on node Worker1_SRI
net.core.wmem_max = 536870912
net.core.rmem_max = 536870912


In [17]:
# set 9k MTU on dataplane interfaces
mtu=9000

for site_name, site_slivers in slivers_by_site.items():
    site_net = site_slivers['net']
    for node in site_slivers['nodes']:
        data_iface = node.get_interface(network_name=site_net.get_name())
        data_iface_name = data_iface.get_os_interface()
        execute_single_node(node, [f"sudo ip link set dev {data_iface_name} mtu {mtu}"])

	Executing "sudo ip link set dev enp7s0 mtu 9000" on node Sender_SRI
	Executing "sudo ip link set dev enp7s0 mtu 9000" on node Worker1_SRI


In [18]:
# run a test from every node to the LB node
for site_name, site_slivers in slivers_by_site.items():
    for node in site_slivers['nodes']:
        print(f'Node {node.get_name()} pinging {lb_node_ip}')
        #execute_single_node(node, [f"sudo ping -q -f -s 8972 -c 100 -M do {lb_node_ip}"])
        execute_single_node(node, [f"sudo ping -q -f -c 100 {lb_node_ip}"])

Node Sender_SRI pinging 10.133.10.254
	Executing "sudo ping -q -f -c 100 10.133.10.254" on node Sender_SRI
PING 10.133.10.254 (10.133.10.254) 56(84) bytes of data.

--- 10.133.10.254 ping statistics ---
100 packets transmitted, 100 received, 0% packet loss, time 1659ms
rtt min/avg/max/mdev = 68.677/68.724/69.226/0.056 ms, pipe 5, ipg/ewma 16.759/68.717 ms
Node Worker1_SRI pinging 10.133.10.254
	Executing "sudo ping -q -f -c 100 10.133.10.254" on node Worker1_SRI
PING 10.133.10.254 (10.133.10.254) 56(84) bytes of data.

--- 10.133.10.254 ping statistics ---
100 packets transmitted, 100 received, 0% packet loss, time 1661ms
rtt min/avg/max/mdev = 68.681/68.739/69.385/0.067 ms, pipe 5, ipg/ewma 16.781/68.730 ms


## Customize Nodes

Customize node setup by adding E2SAR installation

### Add E2SAR software

In [20]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"chmod go-rwx {sign_key_location}",
    # Meson won't detect boost by merely setting cmake_prefix_path, instead set BOOST_ROOT env variable 
    # for gRPC it is enough to set -Dpkg_config_path option to meson
    f"echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.profile",
    f"echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.bashrc",
]

for node in slice.get_nodes():    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(signing_key, sign_key_location)
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node Sender_SRI
	Executing "chmod go-rwx /home/ubuntu/.ssh/signing" on node Sender_SRI
	Executing "echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.profile" on node Sender_SRI
	Executing "echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.bashrc" on node Sender_SRI
	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node Worker1_SRI
	Executing "chmod go-rwx /home/ubuntu/.ssh/signing" on node Worker1_SRI
	Executing "echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.profile" on node Worker1_SRI
	Executing "echo 'export BOOST_ROOT=/usr/local/ LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.bashrc" on node Worker1_SRI


In [24]:
#set JAVA_HOME enviroment variables
java_home = '/usr/lib/jvm/java-1.17.0-openjdk-amd64' ## probably have to change this depending on OS. Tested on Ubuntu 22.04
commands = [
    f"echo 'export JAVA_HOME={java_home}' >> ~/.profile",
    f"echo 'export JAVA_HOME={java_home}' >> ~/.bashrc",
]
 
execute_commands_on_threads(slice.get_nodes(), commands)
print("Note this is executed in parallel on threads, check the log files for completion")

Note this is executed in parallel on threads, check the log files for completion


In [40]:
# download boost and grpc dependencies from releases
print(e2sar_release_url)
commands = [
    f"wget -q -O e2sar-release.deb {e2sar_release_url}",
    f"sudo dpkg -i ./e2sar-release.deb",
]
for node in slice.get_nodes():
    execute_commands(node, commands)

https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb
	Executing "wget -q -O e2sar-release.deb https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb" on node Sender_SRI
	Executing "sudo dpkg -i ./e2sar-release.deb" on node Sender_SRI
Selecting previously unselected package e2sar.
(Reading database ... 96884 files and directories currently installed.)
Preparing to unpack ./e2sar-release.deb ...
Unpacking e2sar (0.1.5) ...
Setting up e2sar (0.1.5) ...
Processing triggers for man-db (2.10.2-1) ...
	Executing "wget -q -O e2sar-release.deb https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb" on node Worker1_SRI
	Executing "sudo dpkg -i ./e2sar-release.deb" on node Worker1_SRI
Selecting previously unselected package e2sar.
(Reading database ... 95601 files and directories currently installed.)
Preparing to unpack ./e2sar-release.d

# Downloading ERSAP and E2SAR java libraries

In [6]:
ersap_home = '$HOME/ersap-install'
commands = [
    f"echo 'export ERSAP_HOME={ersap_home}' >> ~/.profile",
    f"echo 'export ERSAP_HOME={ersap_home}' >> ~/.bashrc",
]
execute_commands_on_threads(slice.get_nodes(), commands)

In [51]:
commands = [
    f"mkdir ERSAP",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b upgradeGradle git@github.com:JeffersonLab/ersap-java.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-e2sar.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/e2sar-java.git",
]
execute_commands_on_threads(slice.get_nodes(), commands)

	Executing "mkdir ERSAP" on node Sender_SRI
 mkdir: cannot create directory ‘ERSAP’: File exists
 	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b upgradeGradle git@github.com:JeffersonLab/ersap-java.git" on node Sender_SRI
 fatal: destination path 'ersap-java' already exists and is not an empty directory.
 	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-e2sar.git" on node Sender_SRI
 fatal: destination path 'ersap-e2sar' already exists and is not an empty directory.
 	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recur

## Installing erap-java
ERSAP_HOME for this test is $HOME/ersap-install

In [43]:
commands = [ 
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-java; ./gradlew deploy",
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-java; ./gradlew publishToMavenLocal",
]
execute_commands_on_threads(slice.get_nodes(), commands)

## Building and installing E2SAR-JAVA

In [45]:
#Building the JNI Shared library
#Might have to set PKG_CONFIG_PATH for cmake build to work. Works without pkg_config_path on 22.04
commands = [
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} JAVA_HOME={java_home} cmake -DCMAKE_INSTALL_PREFIX={ersap_home}/plugins/jni/ -S . -B build",
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} JAVA_HOME={java_home} sudo cmake --build build --target install",
]
execute_commands_on_threads(slice.get_nodes(), commands)

In [47]:
#Compiling Java code and packaging into jar
##IMPORTANT MAVEN ARGUMENTS NEEDED FOR TESTS
# Need to specify -Djava.library.path for linking with the jnie2sar.so built in the step above. Maven surefire tests do not pick this up directly so it has to be encapsulated as -DargLine='-Djava.library.path=build/'
# -Dtest="" is needed to specify class/package of tests with pattern. If this is not specified then all classes with *TEST* will be run by maven surefire

commands = [
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64 mvn -DargLine='-Djava.library.path=${{env.ERSAP_HOME}}/plugins/jni/lib' clean install -Dtest='org.jlab.hpdf.unit.**'",
]

execute_commands_on_threads(slice.get_nodes(), commands)

In [48]:
##Copying build jar to ERSAP_HOME/lib 
commands = [
    f"export ERSAP_HOME={ersap_home}; cp ~/.m2/repository/org/jlab/hpdf/e2sar-java/0.0.1/e2sar-java-0.0.1-jar-with-dependencies.jar $ERSAP_HOME/lib ",
]

execute_commands_on_threads(slice.get_nodes(), commands)


## Installing ersap-e2sar-java (Pure JAVA pipeline with JNI for E2SAR)

In [5]:
commands = [ 
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-e2sar/segmentorJni; ./gradlew install",
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-e2sar/reassemblerJni; ./gradlew install",
]
execute_commands_on_threads(slice.get_nodes(), commands)

NameError: name 'ersap_home' is not defined

## Run Tests

### Run simple single-threaded performance test

Start Segmenter on Sender node and one Reassembler on Worker1 and test throughput **without a real load balancer**. Reassembler is told that LB header will be present and it ignores it.

### Reserve the Load Balancer

In [7]:
sender = list(filter(lambda n: n.get_name()[0:6] == "Sender", slice.get_nodes()))[0]
recver = list(filter(lambda n: n.get_name()[0:7] == "Worker1", slice.get_nodes()))[0]

sender_addr = sender.get_interface(network_name=make_net_name(sender.get_site())).get_ip_addr()
recver_addr = recver.get_interface(network_name=make_net_name(recver.get_site())).get_ip_addr()
print(f"Sender sending from {sender_addr}, receiver receiving on {recver_addr}")

Sender sending from 10.141.132.254, receiver receiving on 10.141.132.253


In [9]:
# ejfat admin URI is set in the preamble

ld_library_path = "LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64"
bin_path = "PATH=/usr/local/bin:$PATH"
# note we are forcing IPv4 here with -4 option - from FABRIC this is necessary
lbadm = f"{ld_library_path} {bin_path} lbadm -4"


In [10]:
# run an overview command to see what is reserved
# we use sender node but any node can be used for admin commands

command = f"{lbadm} --overview -u {ejfat_admin_uri}"

execute_commands(sender, [command])

	Executing "LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64 PATH=/usr/local/bin:$PATH lbadm -4 --overview -u ejfats://COU7L26VIFSVWRVB13S32Z9JDC3DUU6V@10.133.10.254:18008/" on node Sender_SRI
E2SAR Version: 0.1.5
Getting Overview 
   Contacting: ejfats://10.133.10.254:18008/ using address: ipv4:///10.133.10.254:18008


In [11]:
# Reserve the load balancer
lbname = 'ersap-e2sar-testlb'
duration = '02:00:00' # 2 hours

command = f"{lbadm} --reserve -l {lbname} -a '{sender_addr}' -d {duration} -u '{ejfat_admin_uri}' -e"
execute_commands(sender, [command])

	Executing "LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64 PATH=/usr/local/bin:$PATH lbadm -4 --reserve -l ersap-e2sar-testlb -a '10.141.132.254' -d 02:00:00 -u 'ejfats://COU7L26VIFSVWRVB13S32Z9JDC3DUU6V@10.133.10.254:18008/' -e" on node Sender_SRI
export EJFAT_URI='ejfats://8b5d5fb32ec8c55f20644d6c3a6eb5eb8ac2aaaacc4e187c78bffda954bc3cab@10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10]'


In [12]:
# copy the 'Updated URI after reserve with instance token' from the above result here:
instance_uri = 'ejfats://8b5d5fb32ec8c55f20644d6c3a6eb5eb8ac2aaaacc4e187c78bffda954bc3cab@10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10]'

In [13]:
# get the status of the reserved LB (as a check)
command = f"{lbadm} --status -u '{instance_uri}'"
execute_commands(sender, [command])

	Executing "LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64 PATH=/usr/local/bin:$PATH lbadm -4 --status -u 'ejfats://8b5d5fb32ec8c55f20644d6c3a6eb5eb8ac2aaaacc4e187c78bffda954bc3cab@10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10]'" on node Sender_SRI
E2SAR Version: 0.1.5
Getting LB Status 
   Contacting: ejfats://10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10] using address: ipv4:///10.133.10.254:18008
   LB ID: 7
Registered sender addresses: 10.141.132.254 
Registered workers: 
LB details: expiresat=2025-02-17T16:21:41Z, currentepoch=0, predictedeventnum=18446744073709551615


### Update yaml files for test

In [15]:
#yq e -i '.configuration.io-services.reader.events = 100' config/live/services.yaml 
num_events = 100
event_size = 80000
commands = [ 
    f"cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.reader.events = {num_events}' config/live/byte_services.yaml ",
    f"cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.reader.eventSize = {event_size}' config/live/byte_services.yaml",
]
execute_commands(sender, commands)

	Executing "cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.reader.events = 100' config/live/byte_services.yaml " on node Sender_SRI
	Executing "cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.reader.eventSize = 80000' config/live/byte_services.yaml" on node Sender_SRI


In [20]:
commands = [ 
    f"cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.writer.EJFAT_URI = \"{instance_uri}\"' config/live/byte_services.yaml ",
    f"cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.writer.SENDER_IP = \"{sender_addr}\"' config/live/byte_services.yaml",
]
execute_commands(sender, commands)

	Executing "cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.writer.EJFAT_URI = "ejfats://8b5d5fb32ec8c55f20644d6c3a6eb5eb8ac2aaaacc4e187c78bffda954bc3cab@10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10]"' config/live/byte_services.yaml " on node Sender_SRI
	Executing "cd ERSAP/ersap-e2sar/segmentorJni; yq e -i '.configuration.io-services.writer.SENDER_IP = "10.141.132.254"' config/live/byte_services.yaml" on node Sender_SRI


In [22]:
commands = [ 
    f"cd ERSAP/ersap-e2sar/reassemblerJni; yq e -i '.configuration.io-services.reader.EJFAT_URI = \"{instance_uri}\"' config/live/services.yaml ",
    f"cd ERSAP/ersap-e2sar/reassemblerJni; yq e -i '.configuration.io-services.reader.WORKER_IP = \"{recver_addr}\"' config/live/services.yaml",
]
execute_commands(recver, commands)

	Executing "cd ERSAP/ersap-e2sar/reassemblerJni; yq e -i '.configuration.io-services.reader.EJFAT_URI = "ejfats://8b5d5fb32ec8c55f20644d6c3a6eb5eb8ac2aaaacc4e187c78bffda954bc3cab@10.133.10.254:18008/lb/7?sync=10.133.10.254:19010&data=10.133.10.253&data=[2001:400:a300::10]"' config/live/services.yaml " on node Worker1_SRI
	Executing "cd ERSAP/ersap-e2sar/reassemblerJni; yq e -i '.configuration.io-services.reader.WORKER_IP = "10.141.132.253"' config/live/services.yaml" on node Worker1_SRI


### Run a test with real load balancer and single sender node and single receiver node

I intentionally left out the code to run the segmenter and reassembler pipeline from this notebook. The reassembler pipeline will indefinitely wait for events until a SIGINT(ctrl+c). You need to ssh into the sender and worker nodes to start the pipelines. Run the reassembler pipeline first so it is able to receive events before the segmenter


For the recevr the command will be cd ERSAP/ersap-e2sar/reassemblerJNI; \\$ERSAP_HOME/bin/ersap-shell config/live/reassembler_live.yaml\
For the sender the command will be cd ERSAP/ersap-e2sar/segmentorJni; \\$ERSAP_HOME/bin/ersap-shell config/live/segmentor_live.yaml


In [ ]:
# free the load balancer
command = f"{lbadm} --free -u '{instance_uri}'"

execute_commands(sender, [command])

## Manage the slice

### Extend by two weeks

In [5]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")


Retry: 0, Time: 36 sec


ID,5210f023-b755-498a-ba3a-956056527da8
Name,4-node U280 LB Tester Slice using ubuntu22 1
Lease Expiration (UTC),2025-02-24 01:30:08 +0000
Lease Start (UTC),2025-01-16 21:08:36 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
d712ad68-da39-4dcc-a38d-25ccadb49867,Sender_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.83,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.83,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
972b4bf3-ed61-4be1-b976-849342753474,Worker1_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.108,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.108,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1df99571-43d1-4afb-bc5a-a164f22f2516,Worker2_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.234,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.234,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
06aa6397-8d79-4c05-b3d2-6d1ec510d3c2,Worker3_SRI,8,32,100,default_ubuntu_22,qcow2,sri-w2.fabric-testbed.net,SRI,ubuntu,192.5.67.106,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.106,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
d9f34c4b-8736-426e-bcad-d6c341700282,fabnetv4_SRI,L3,FABNetv4,SRI,10.141.136.0/24,10.141.136.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Sender_SRI-Sender_SRI_NIC_Basic_nic-p1,p1,Sender_SRI,fabnetv4_SRI,100,config,,02:87:0F:34:90:FD,enp7s0,enp7s0,10.141.136.252,4,HundredGigE0/0/0/9
Worker1_SRI-Worker1_SRI_NIC_Basic_nic-p1,p1,Worker1_SRI,fabnetv4_SRI,100,config,,06:FE:1D:62:9A:19,enp7s0,enp7s0,10.141.136.254,4,HundredGigE0/0/0/9
Worker2_SRI-Worker2_SRI_NIC_Basic_nic-p1,p1,Worker2_SRI,fabnetv4_SRI,100,config,,0A:6F:F3:10:A5:CD,enp7s0,enp7s0,10.141.136.251,4,HundredGigE0/0/0/9
Worker3_SRI-Worker3_SRI_NIC_Basic_nic-p1,p1,Worker3_SRI,fabnetv4_SRI,100,config,,0A:AC:B9:5A:58:A5,enp7s0,enp7s0,10.141.136.253,4,HundredGigE0/0/0/9



Time to print interfaces 48 seconds


### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()